
---

## 🔍 What is a "Run Type" in LangSmith?

A **run** in LangSmith represents a **step** in the execution of your GenAI application — this could be calling an LLM, retrieving documents, parsing a response, etc.

LangSmith classifies each of these runs by **type** to help:

* Visualize trace trees meaningfully
* Debug parts more easily
* Group similar runs for performance/cost analysis

---

## 🧩 1. **LLM Run**

### 📘 **What it is**:

A run that represents a **Large Language Model** (LLM) call — like OpenAI, Anthropic, Mistral, etc.

### 🧠 **Use Case**:

Every time you **call an LLM to generate** output (e.g., summary, response, SQL generation), LangSmith creates an **LLM-type run**.

### 🔧 **How it looks in LangSmith**:

* Clearly labeled as `LLM`
* Shows input messages or prompts
* Shows temperature, model name, tokens, latency

### 📍 **Example**:

```python
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-4")
llm.predict("Summarize the following article...")
```

This creates a **LLM Run** in the trace tree under your main Chain.

---

## 📚 2. **Retriever Run**

### 📘 **What it is**:

This represents a **retrieval call** to fetch documents from a vector database or search index (like FAISS, Pinecone, Weaviate, etc.)

### 🧠 **Use Case**:

In **RAG (Retrieval-Augmented Generation)**, this run tells you what docs were fetched and why.

### 🔧 **How LangSmith handles it**:

* Shows query passed
* Shows the list of retrieved documents
* Time taken to retrieve

### 📍 **Example**:

```python
from langchain.vectorstores import FAISS

retriever = faiss.as_retriever()
retriever.get_relevant_documents("What is LangSmith?")
```

LangSmith creates a **Retriever Run**, showing:

* Query string
* Retrieved docs
* Embedding model info (if passed via metadata)

---

## 🛠️ 3. **Tool Run**

### 📘 **What it is**:

A **Tool Run** occurs when an **Agent calls a tool** during decision-making.

### 🧠 **Use Case**:

When using LangChain’s Agent architecture, the agent often decides to call tools like:

* Calculator
* Search
* Weather
* Custom API tools

### 🔧 **How LangSmith visualizes it**:

* Shows **input to tool**
* Shows **output from tool**
* Captures **tool name**

### 📍 **Example**:

```python
from langchain.agents import initialize_agent, Tool

tools = [Tool(name="Calculator", func=my_calc_func, description="...")]
agent = initialize_agent(tools=tools, llm=llm, agent_type="zero-shot-react-description")

agent.run("What is 5 * 11?")
```

You’ll see:

* `Tool Run` for `Calculator`
* Input: `"5 * 11"`
* Output: `"55"`

---

## ⛓️ 4. **Chain Run** (Default Type)

### 📘 **What it is**:

Any **LangChain chain or function** you decorate with `@traceable` becomes a **Chain run** by default.

### 🧠 **Use Case**:

* You write a function that glues other parts: retrieval + LLM + formatting → that's a chain.
* This is often the **root** of your trace.

### 🔧 **Why Important**:

Chains are the **structural units** in your trace tree — they often contain:

* LLM Runs
* Retriever Runs
* Prompt Runs
* Tool Runs

### 📍 **Example**:

```python
from langsmith.traceable import traceable

@traceable(name="answer_question_chain")
def answer_question(user_input: str) -> str:
    docs = retriever.get_relevant_documents(user_input)
    return llm.predict(f"Answer based on: {docs}")
```

This entire logic gets traced as a **Chain Run**, which contains nested **Retriever + LLM runs**.

---

## 📝 5. **Prompt Run**

### 📘 **What it is**:

A **Prompt Run** represents the stage where a **prompt template is being filled or formatted** before being sent to the LLM.

### 🧠 **Use Case**:

You might have complex templates with variables. If something goes wrong, prompt run helps debug it.

### 🔧 **What it captures**:

* Template structure
* Final filled prompt
* Variables used

### 📍 **Example**:

```python
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("Answer based on: {context}\nQuestion: {question}")
formatted = prompt.format(context="Earth is round", question="What is Earth’s shape?")
```

LangSmith records:

* Template: `"Answer based on: {context}\nQuestion: {question}"`
* Final Prompt: `"Answer based on: Earth is round\nQuestion: What is Earth’s shape?"`

Useful for prompt debugging!

---

## 🔄 6. **Parser Run**

### 📘 **What it is**:

When you **parse** the raw output from an LLM (JSON, text, SQL), LangSmith can log it as a **Parser Run**.

### 🧠 **Use Case**:

* LLM outputs structured data
* You extract key fields
* If parsing fails, LangSmith shows it clearly

### 🔧 **Typical scenario**:

* Parsing with `StructuredOutputParser`
* Custom parsing logic

### 📍 **Example**:

```python
from langchain.output_parsers import StructuredOutputParser

parser = StructuredOutputParser.from_response_schema(schema)
result = parser.parse(llm_output)
```

LangSmith captures:

* Raw LLM output
* Parsing function name
* Final structured result

If parsing fails (e.g., invalid JSON), you can debug **directly in the trace**.

---

## 🔁 Summary Table

| **Run Type** | **What It Traces**                        | **Why It’s Useful**                                     |
| ------------ | ----------------------------------------- | ------------------------------------------------------- |
| `LLM`        | LLM generation calls                      | See prompts, responses, cost, and latency               |
| `Retriever`  | Document retrieval logic                  | Debug RAG performance, trace missing docs               |
| `Tool`       | Agent tool calls                          | Monitor tool decisions and outputs                      |
| `Chain`      | Composite logic (default with @traceable) | Structure the full flow, contains nested runs           |
| `Prompt`     | Template formatting                       | Ensure prompt accuracy and debug inputs                 |
| `Parser`     | Parsing model outputs                     | Trace data transformation, spot malformed output issues |

---

## 🔍 Case Study: Full Trace Tree

```
Root (Chain): answer_question_chain
├── Prompt: filled user + system prompt
├── Retriever: get_relevant_documents(query)
│   └── Metadata: {"retriever": "Weaviate"}
├── LLM: OpenAI GPT-4
│   └── Input: Prompt text
│   └── Output: Model answer
└── Parser: JSON to dict (StructuredOutputParser)
```

---

## ✅ Must-Know Questions

1. What are the 6 run types in LangSmith and their roles?
2. What run type does `@traceable` default to, and how can you customize it?
3. How do nested run types help you debug a complex agent?
4. If your RAG pipeline is not returning documents, which run type should you inspect?
5. How would you debug prompt formatting issues using LangSmith?
6. How does a parser run help avoid silent failures in structured output?

---



---

### ✅ **1. What are the 6 run types in LangSmith and their roles?**

| **Run Type**  | **Role**                                                                                                   |
| ------------- | ---------------------------------------------------------------------------------------------------------- |
| **LLM**       | Traces calls to a language model (e.g., GPT-4, Claude) to generate text.                                   |
| **Retriever** | Traces document retrieval logic — when documents are pulled from vector DBs or search engines.             |
| **Tool**      | Traces external tools used by agents, e.g., calculator, search tool, or custom APIs.                       |
| **Chain**     | Represents a composite logical unit like a pipeline — default for `@traceable`-decorated functions.        |
| **Prompt**    | Captures prompt template rendering — i.e., what was the final prompt given to the LLM.                     |
| **Parser**    | Captures output parsing logic — e.g., converting JSON strings or structured responses into usable objects. |

📘 **Why important?**
LangSmith categorizes these run types so developers can isolate and debug individual stages in a multi-step GenAI pipeline.

---

### ✅ **2. What run type does `@traceable` default to, and how can you customize it?**

* By default, `@traceable` logs a run as type **`chain`**.
* This makes sense because a decorated function usually wraps a *pipeline* or composite step.

#### 🛠️ Customize it like this:

```python
@traceable(name="parse_tool_output", run_type="parser")
def parse_output(output):
    ...
```

✔️ This flexibility allows you to **accurately represent the function’s intent** in your trace tree, improving readability and debugging.

---

### ✅ **3. How do nested run types help you debug a complex agent?**

When using LangSmith, **nested runs** give you a **call hierarchy**.

Imagine a multi-step agent:

```
Root Chain (Agent Execution)
├── Tool Run (Search API)
│   └── LLM Run (Rephrasing the search result)
├── Tool Run (Calculator)
├── LLM Run (Final synthesis)
```

With nesting:

* You **see the tool calls** and their responses.
* You **see which LLM calls** are being made at which steps.
* You **track latency per component**.

✅ This helps isolate bugs like:

* Tool call failed? Go to Tool Run.
* Response format broke? Go to Parser Run.
* Wrong reasoning? Go to LLM Run.

---

### ✅ **4. If your RAG pipeline is not returning documents, which run type should you inspect?**

👉 You should **inspect the `Retriever` run**.

This will show:

* What query was passed to the retriever
* What documents were returned (if any)
* If there was a timeout or API error
* Metadata like retriever type or vector DB used

#### 📍 Example:

```python
@traceable
def retrieve(user_query):
    return retriever.get_relevant_documents(user_query)
```

🧠 If zero documents are returned, maybe:

* The embedding is off
* The query is poorly formatted
* The DB is misconfigured

LangSmith helps by **logging all of this visibly** under the Retriever Run node.

---

### ✅ **5. How would you debug prompt formatting issues using LangSmith?**

You’d inspect the **Prompt Run**.

It will show:

* The prompt **template** (with placeholders)
* The final **rendered prompt** (after variable substitution)
* All **inputs passed** for formatting

#### 📍 Example:

```python
Prompt Template:
"Answer using context: {context}\nQuestion: {question}"

Filled Prompt:
"Answer using context: \nQuestion: What is LangSmith?"
```

🤯 A-ha! `context` was empty — which is why the LLM hallucinated. Without prompt runs, this would be hard to catch.

---

### ✅ **6. How does a parser run help avoid silent failures in structured output?**

LLMs often return **structured responses** like JSON. If the format is invalid and your parser crashes, you may not catch the bug unless it's logged.

The **Parser Run**:

* Traces your parsing logic
* Captures the **raw LLM output**
* Captures the **parsed result**
* Logs any **errors** (e.g., `JSONDecodeError`)

#### 📍 Example:

```python
@traceable(run_type="parser")
def parse_json_response(text):
    return json.loads(text)
```

LangSmith will show:

* Input: `'{"answer": "42"}'`
* Output: `{"answer": "42"}`

Or, if LLM returns: `"The answer is 42"` → ❌ JSON parsing fails
👉 You now know exactly **where** and **why** the error occurred.

---

## ✅ Final Summary of All Answers

| **Question**     | **Main Takeaway**                                                              |
| ---------------- | ------------------------------------------------------------------------------ |
| Run types        | LLM, Retriever, Tool, Chain, Prompt, Parser — each has a clear debugging role. |
| Default run type | `@traceable` = Chain by default, can be customized.                            |
| Nested runs      | Help trace and debug complex agents or RAG pipelines.                          |
| RAG bug?         | Check `Retriever` run to debug document fetch issues.                          |
| Prompt bug?      | Check `Prompt` run to inspect final rendered prompts.                          |
| Parsing bug?     | Use `Parser` run to debug structured output issues.                            |

---
